In [1]:
%%bash
git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
cd rubiks-cube-NxNxN-solver
make init

Cloning into 'rubiks-cube-NxNxN-solver'...


rm -rf build dist venv rubikscubennnsolver.egg-info cache ida_search ida_search_via_graph my-pt-states.txt
find . -name __pycache__ | xargs rm -rf
gcc -O3 -o ida_search_via_graph rubikscubennnsolver/ida_search_core.c rubikscubennnsolver/rotate_xxx.c rubikscubennnsolver/ida_search_666.c rubikscubennnsolver/ida_search_777.c rubikscubennnsolver/ida_search_via_graph.c -lm
python3 -m venv venv
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.8 MB/s eta 0:00:0031m30.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///home/borys/santa-2023/dwalton76/rubiks-cube-NxNxN-solver (from -r requirements.dev.txt (line 10))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done

In [3]:
%%bash
cd ..
git clone https://github.com/dwalton76/kociemba.git
cd kociemba/kociemba/ckociemba/
make
make install

fatal: destination path 'kociemba' already exists and is not an empty directory.


mkdir -p bin
gcc -std=c99 -O3 -D_XOPEN_SOURCE=700 coordcube.c cubiecube.c facecube.c search.c -Iinclude solve.c -o bin/kociemba
cp bin/kociemba /usr/local/bin/


cp: cannot create regular file '/usr/local/bin/kociemba': Permission denied
make: *** [Makefile:16: install] Error 1


CalledProcessError: Command 'b'cd ..\ngit clone https://github.com/dwalton76/kociemba.git\ncd kociemba/kociemba/ckociemba/\nmake\nmake install\n'' returned non-zero exit status 2.

In [4]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

os.chdir('rubiks-cube-NxNxN-solver')
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /home/borys/santa-2023/dwalton76/rubiks-cube-NxNxN-solver


In [5]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}
def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]

def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M



In [19]:
p = '/home/borys/santa-2023/data/'
puzzles = pd.read_csv(p + 'puzzles_ab.csv', index_col='id')
puzzle_info = pd.read_csv(p + 'puzzle_info.csv', index_col='puzzle_type')
submission = pd.read_csv(p + 'sample_submission.csv', index_col='id')

In [20]:
outputs = {}
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(30, 284):
            continue
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = move_translation(dim)

        init_state = row['initial_state']
        sol_state = row['solution_state']

        # Only attempt to solve cubes with traditional solutions. 
        # That is, cubes that have a solution state where each face 
        # filled with equivilent letters/colors.
        if True:
            state = state2ubl(init_state)
            print(f'Starting {id}')
            output = !/home/borys/santa-2023/dwalton76/rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state $state
            outputs[id] = output
            if output[-1][:9] == 'Solution:':
                sol = output[-1].split(': ')[1]
            else:
                for n in range(1, 21):
                    if 'Solution:' in output[-n]:
                        sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                        break
                
            mmoves = '.'.join([M[m] for m in sol.split(' ')])
            
            new_state = init_state
            for move in mmoves.split('.'):
                new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
            
            I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
            manipulations = [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for
                                                                              i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
            for init_moves in manipulations:
                temp_state = new_state
                if len(init_moves) > 0:
                    for move in init_moves.split('.'):
                        temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
                if temp_state == sol_state:
                    print(f'solved id: {id}')
                    if len(init_moves) > 0:
                        mmoves += '.' + init_moves
                    break
                    
            # validation
            state = row['initial_state'].split(";")
            for move_name in mmoves.split('.'):
                state = [state[i] for i in moves[move_name]]

            try:
                
                assert row['solution_state'].split(";") == state
                
                mmoves_length = len(mmoves.split('.'))
                best_moves_length = len(submission.loc[id, 'moves'].split('.')) 
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    submission.loc[id, 'moves'] = mmoves
            except AssertionError:
                print(f"assertion error for {id}")
                print(row['solution_state'])
                print(';'.join(state))
                break

Starting 130
solved id: 130
Starting 131
solved id: 131
Starting 132
solved id: 132
Starting 133
solved id: 133
Starting 134
solved id: 134
Starting 135
solved id: 135
Starting 136
solved id: 136
Starting 137
solved id: 137
Starting 138
solved id: 138
Starting 139
solved id: 139
Starting 235
solved id: 235
Starting 236
solved id: 236
Starting 237
solved id: 237
Starting 238
solved id: 238
Starting 239
solved id: 239
Starting 282
solved id: 282


In [17]:
print(f"Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")

Score: 1151262


In [18]:
submission.reset_index().to_csv('../sub_ab.csv', index=False)